In [1]:
from heeps.util.multiCPU import multiCPU
from heeps.util.freq_decomp import remove_zernike
from heeps.util.img_processing import resize_cube
import os.path
import proper
from copy import deepcopy

In [2]:
rep = '/mnt/disk4tb/METIS/METIS_CBASIC_CUBES/water_vapor/'
scao = rep + '../cube_Cbasic_20201130_3600s_300ms_0piston_meters_%s_285.fits'
wv = rep + 'cube_Cbasic_20210601_3600s_300ms_0piston_meters_scao_only_720_WV.fits'
pup = fits.getdata(rep + 'mask_Cbasic_20210601_720.fits')
pup[pup < .5] = 0
n1 = fits.getdata(rep + 'cube_Cbasic_20210601_3600s_100ms_0piston_meters_scao_only_720_WV_zpols_100_nLF_1_nHF_1_G_0.4.fits')[::3]
n3 = fits.getdata(rep + 'cube_Cbasic_20210601_3600s_100ms_0piston_meters_scao_only_720_WV_zpols_100_nLF_3_nHF_3_G_0.4.fits')[::3]
n10 = fits.getdata(rep + 'cube_Cbasic_20210601_3600s_100ms_0piston_meters_scao_only_720_WV_zpols_100_nLF_10_nHF_10_G_0.4.fits')[::3]
nimg = 720
npupil = 285
temporal_rms = 8814.11
rms_L = 100
rms_N = 1200

In [3]:
wf = proper.prop_begin(nimg, 1, nimg, 1) # initial wavefront
name = rep + 'cube_Cbasic_20210601_3600s_300ms_0piston_meters_%s_285_WV%s_G_0.4_qacits_%s.fits'
for freq, zern in zip([1,3,10], [n1[:,:3], n3[:,:3], n10[:,:3]]):
    if os.path.isfile(name%('all_ncpa', 'N', freq)):
        print('file exists: ' + name%('all_ncpa', 'N', freq))
    else:
        _, HSF = multiCPU(remove_zernike, posargs=[deepcopy(wf), pup], \
            posvars=[fits.getdata(wv), zern], \
            case='qacits n%s'%freq, nout=2, verbose=True)
        HSF = resize_cube(HSF, npupil)
        for rms, band in zip([rms_L, rms_N], ['L', 'N']):
            for case in ['scao_only', 'all_ncpa']:
                print('writeto ' + name%(case, band, freq))
                fits.writeto(name%(case, band, freq), np.float32(fits.getdata(scao%case) + HSF*(rms/temporal_rms)), overwrite=True)

file exists: /mnt/disk4tb/METIS/METIS_CBASIC_CUBES/water_vapor/cube_Cbasic_20210601_3600s_300ms_0piston_meters_all_ncpa_285_WVN_G_0.4_qacits_1.fits
file exists: /mnt/disk4tb/METIS/METIS_CBASIC_CUBES/water_vapor/cube_Cbasic_20210601_3600s_300ms_0piston_meters_all_ncpa_285_WVN_G_0.4_qacits_3.fits
file exists: /mnt/disk4tb/METIS/METIS_CBASIC_CUBES/water_vapor/cube_Cbasic_20210601_3600s_300ms_0piston_meters_all_ncpa_285_WVN_G_0.4_qacits_10.fits
